In [ ]:
!git clone https://github.com/Plachtaa/seed-vc.git

In [ ]:
%pip install -r "C:\Users\Welcome\OneDrive\Documents\seed-vc\requirements.txt"

In [1]:
%pip install streamlit pyngrok demucs pydub

  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0+cu121
    Uninstalling torch-2.4.0+cu121:
      Successfully uninstalled torch-2.4.0+cu121
  Rolling back uninstall of torch
  Moving to c:\users\welcome\appdata\roaming\python\python312\site-packages\functorch\
   from C:\Users\Welcome\AppData\Roaming\Python\Python312\site-packages\~unctorch
  Moving to c:\users\welcome\appdata\roaming\python\python312\site-packages\torch-2.4.0+cu121.dist-info\
   from C:\Users\Welcome\AppData\Roaming\Python\Python312\site-packages\~orch-2.4.0+cu121.dist-info
  Moving to c:\users\welcome\appdata\roaming\python\python312\site-packages\torch\__config__.py
   from C:\Users\Welcome\AppData\Lo

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\ProgramData\\anaconda3\\Lib\\site-packages\\torch\\lib\\uv.dll'
Consider using the `--user` option or check the permissions.



In [2]:
%cd "C:\Users\Welcome\OneDrive\Documents\seed-vc"
%ls

C:\Users\Welcome\OneDrive\Documents\seed-vc
 Volume in drive C has no label.
 Volume Serial Number is C07F-8A6B

 Directory of C:\Users\Welcome\OneDrive\Documents\seed-vc

27-03-2025  11:18    <DIR>          .
27-03-2025  09:09    <DIR>          ..
27-03-2025  09:11               250 .gitignore
27-03-2025  11:18    <DIR>          __pycache__
27-03-2025  09:11            19,528 app.py
27-03-2025  09:11            23,226 app_svc.py
27-03-2025  09:11            20,212 app_vc.py
27-03-2025  09:11    <DIR>          assets
27-03-2025  09:11    <DIR>          baselines
27-03-2025  09:11        28,036,335 campplus_cn_common.bin
27-03-2025  09:11               407 conda-nix-vc-py310.yaml
27-03-2025  09:11    <DIR>          configs
27-03-2025  09:11    <DIR>          dac
27-03-2025  09:11    <DIR>          data
27-03-2025  09:11            11,395 EVAL.md
27-03-2025  09:11            24,930 eval.py
27-03-2025  09:11    <DIR>          examples
27-03-2025  09:11               520 hf_utils.py
27-03-

In [3]:
%%writefile streamlit_app.py
import os
import streamlit as st
import torch
import torchaudio
import librosa
import numpy as np
import yaml
from io import BytesIO
import tempfile
from pydub import AudioSegment
import subprocess
import uuid
import tempfile
import shutil

# Configure page
st.set_page_config(
    page_title="Voice Conversion Studio",
    page_icon="🎤",
    layout="wide",
)

# Apply custom CSS for black and white theme
st.markdown("""
<style>
    .main {
        background-color: #121212;
        color: #FFFFFF;
    }
    .stButton>button {
        background-color: #FFFFFF;
        color: #121212;
        border: 1px solid #FFFFFF;
        border-radius: 4px;
        padding: 0.5em 1em;
        font-weight: bold;
    }
    .stButton>button:hover {
        background-color: #121212;
        color: #FFFFFF;
        border: 1px solid #FFFFFF;
    }
    .stSlider>div>div {
        background-color: #444444;
    }
    .stSlider>div>div>div>div {
        background-color: #FFFFFF;
    }
    h1, h2, h3 {
        color: #FFFFFF;
    }
    .stHeader {
        background-color: #000000;
        padding: 1em;
        border-radius: 5px;
        margin-bottom: 1em;
    }
    .stInfo {
        background-color: #1E1E1E;
        padding: 1em;
        border-radius: 5px;
        border: 1px solid #444444;
    }
    .stExample {
        background-color: #1E1E1E;
        padding: 1em;
        border-radius: 5px;
        margin-bottom: 0.5em;
        border: 1px solid #333333;
    }
    .stExample:hover {
        border: 1px solid #FFFFFF;
    }
</style>
""", unsafe_allow_html=True)

# Set environment variables
os.environ['HF_HUB_CACHE'] = './checkpoints/hf_cache'

@st.cache_resource
def load_models(use_fp16=True):
    """
    Load and cache all required models for voice conversion
    """
    if torch.cuda.is_available():
        device = torch.device("cuda")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
    else:
        device = torch.device("cpu")

    st.info(f"Using device: {device}, FP16: {use_fp16}")

    # Import required modules only when loading models
    from modules.commons import build_model, load_checkpoint, recursive_munch
    from hf_utils import load_custom_model_from_hf
    from modules.campplus.DTDNN import CAMPPlus
    from modules.rmvpe import RMVPE
    from modules.audio import mel_spectrogram

    # Load main model
    dit_checkpoint_path, dit_config_path = load_custom_model_from_hf(
        "Plachta/Seed-VC",
        "DiT_seed_v2_uvit_whisper_base_f0_44k_bigvgan_pruned_ft_ema_v2.pth",
        "config_dit_mel_seed_uvit_whisper_base_f0_44k.yml"
    )

    config = yaml.safe_load(open(dit_config_path, "r"))
    model_params = recursive_munch(config["model_params"])
    model_params.dit_type = 'DiT'
    model = build_model(model_params, stage="DiT")
    hop_length = config["preprocess_params"]["spect_params"]["hop_length"]
    sr = config["preprocess_params"]["sr"]

    # Load model checkpoints
    model, _, _, _ = load_checkpoint(
        model, None, dit_checkpoint_path,
        load_only_params=True, ignore_modules=[], is_distributed=False,
    )
    for key in model:
        model[key].eval()
        model[key].to(device)
    model.cfm.estimator.setup_caches(max_batch_size=1, max_seq_length=8192)

    # Load CAMP+ model
    campplus_ckpt_path = load_custom_model_from_hf(
        "funasr/campplus", "campplus_cn_common.bin", config_filename=None
    )
    campplus_model = CAMPPlus(feat_dim=80, embedding_size=192)
    campplus_model.load_state_dict(torch.load(campplus_ckpt_path, map_location="cpu"))
    campplus_model.eval()
    campplus_model.to(device)

    # Load vocoder
    vocoder_type = model_params.vocoder.type
    if vocoder_type == 'bigvgan':
        from modules.bigvgan import bigvgan
        bigvgan_name = model_params.vocoder.name
        bigvgan_model = bigvgan.BigVGAN.from_pretrained(bigvgan_name, use_cuda_kernel=False)
        bigvgan_model.remove_weight_norm()
        bigvgan_model = bigvgan_model.eval().to(device)
        vocoder_fn = bigvgan_model
    else:
        st.error(f"Unsupported vocoder type: {vocoder_type}")
        vocoder_fn = None

    # Load speech tokenizer
    speech_tokenizer_type = model_params.speech_tokenizer.type
    if speech_tokenizer_type == 'whisper':
        from transformers import AutoFeatureExtractor, WhisperModel
        whisper_name = model_params.speech_tokenizer.name
        whisper_model = WhisperModel.from_pretrained(whisper_name, torch_dtype=torch.float16).to(device)
        del whisper_model.decoder
        whisper_feature_extractor = AutoFeatureExtractor.from_pretrained(whisper_name)

        def semantic_fn(waves_16k):
            ori_inputs = whisper_feature_extractor([waves_16k.squeeze(0).cpu().numpy()],
                                                return_tensors="pt",
                                                return_attention_mask=True)
            ori_input_features = whisper_model._mask_input_features(
                ori_inputs.input_features, attention_mask=ori_inputs.attention_mask).to(device)
            with torch.no_grad():
                ori_outputs = whisper_model.encoder(
                    ori_input_features.to(whisper_model.encoder.dtype),
                    head_mask=None,
                    output_attentions=False,
                    output_hidden_states=False,
                    return_dict=True,
                )
            S_ori = ori_outputs.last_hidden_state.to(torch.float32)
            S_ori = S_ori[:, :waves_16k.size(-1) // 320 + 1]
            return S_ori
    else:
        st.error(f"Unsupported speech tokenizer type: {speech_tokenizer_type}")
        semantic_fn = None

    # Setup mel spectrogram function
    mel_fn_args = {
        "n_fft": config['preprocess_params']['spect_params']['n_fft'],
        "win_size": config['preprocess_params']['spect_params']['win_length'],
        "hop_size": config['preprocess_params']['spect_params']['hop_length'],
        "num_mels": config['preprocess_params']['spect_params']['n_mels'],
        "sampling_rate": sr,
        "fmin": config['preprocess_params']['spect_params'].get('fmin', 0),
        "fmax": None if config['preprocess_params']['spect_params'].get('fmax', "None") == "None" else 8000,
        "center": False
    }
    to_mel = lambda x: mel_spectrogram(x, **mel_fn_args)

    # F0 extractor
    model_path = load_custom_model_from_hf("lj1995/VoiceConversionWebUI", "rmvpe.pt", None)
    rmvpe = RMVPE(model_path, is_half=False, device=device)
    f0_fn = rmvpe.infer_from_audio

    # Calculate streaming parameters
    max_context_window = sr // hop_length * 30
    overlap_frame_len = 16
    overlap_wave_len = overlap_frame_len * hop_length

    return {
        "model": model,
        "semantic_fn": semantic_fn,
        "vocoder_fn": vocoder_fn,
        "campplus_model": campplus_model,
        "to_mel": to_mel,
        "f0_fn": f0_fn,
        "sr": sr,
        "hop_length": hop_length,
        "device": device,
        "max_context_window": max_context_window,
        "overlap_frame_len": overlap_frame_len,
        "overlap_wave_len": overlap_wave_len,
        "use_fp16": use_fp16
    }

def adjust_f0_semitones(f0_sequence, n_semitones):
    """Adjust F0 by semitones"""
    factor = 2 ** (n_semitones / 12)
    return f0_sequence * factor

def crossfade(chunk1, chunk2, overlap):
    """Apply crossfade between two audio chunks"""
    fade_out = np.cos(np.linspace(0, np.pi / 2, overlap)) ** 2
    fade_in = np.cos(np.linspace(np.pi / 2, 0, overlap)) ** 2
    chunk2[:overlap] = chunk2[:overlap] * fade_in + chunk1[-overlap:] * fade_out
    return chunk2

@torch.no_grad()
@torch.inference_mode()
def convert_voice(models, source_path, target_path, settings):
    """
    Perform voice conversion with the provided settings
    """
    # Unpack models and parameters
    model = models["model"]
    semantic_fn = models["semantic_fn"]
    vocoder_fn = models["vocoder_fn"]
    campplus_model = models["campplus_model"]
    to_mel = models["to_mel"]
    f0_fn = models["f0_fn"]
    sr = models["sr"]
    hop_length = models["hop_length"]
    device = models["device"]
    max_context_window = models["max_context_window"]
    overlap_frame_len = models["overlap_frame_len"]
    overlap_wave_len = models["overlap_wave_len"]
    use_fp16 = models["use_fp16"]

    # Unpack settings
    diffusion_steps = settings["diffusion_steps"]
    length_adjust = settings["length_adjust"]
    inference_cfg_rate = settings["inference_cfg_rate"]
    auto_f0_adjust = settings["auto_f0_adjust"]
    pitch_shift = settings["pitch_shift"]

    # Load audio files
    source_audio = librosa.load(source_path, sr=sr)[0]
    ref_audio = librosa.load(target_path, sr=sr)[0]

    # Process audio
    source_audio = torch.tensor(source_audio).unsqueeze(0).float().to(device)
    ref_audio = torch.tensor(ref_audio[:sr * 25]).unsqueeze(0).float().to(device)

    # Resample to 16kHz
    ref_waves_16k = torchaudio.functional.resample(ref_audio, sr, 16000)
    converted_waves_16k = torchaudio.functional.resample(source_audio, sr, 16000)

    # Extract semantic features
    if converted_waves_16k.size(-1) <= 16000 * 30:
        S_alt = semantic_fn(converted_waves_16k)
    else:
        # Process longer audio in chunks
        overlapping_time = 5  # 5 seconds
        S_alt_list = []
        buffer = None
        traversed_time = 0
        while traversed_time < converted_waves_16k.size(-1):
            if buffer is None:  # first chunk
                chunk = converted_waves_16k[:, traversed_time:traversed_time + 16000 * 30]
            else:
                chunk = torch.cat([buffer, converted_waves_16k[:, traversed_time:traversed_time + 16000 * (30 - overlapping_time)]], dim=-1)
            S_alt = semantic_fn(chunk)
            if traversed_time == 0:
                S_alt_list.append(S_alt)
            else:
                S_alt_list.append(S_alt[:, 50 * overlapping_time:])
            buffer = chunk[:, -16000 * overlapping_time:]
            traversed_time += 30 * 16000 if traversed_time == 0 else chunk.size(-1) - 16000 * overlapping_time
        S_alt = torch.cat(S_alt_list, dim=1)

    ori_waves_16k = torchaudio.functional.resample(ref_audio, sr, 16000)
    S_ori = semantic_fn(ori_waves_16k)

    # Generate mel spectrograms
    mel = to_mel(source_audio.to(device).float())
    mel2 = to_mel(ref_audio.to(device).float())

    # Set target lengths
    target_lengths = torch.LongTensor([int(mel.size(2) * length_adjust)]).to(mel.device)
    target2_lengths = torch.LongTensor([mel2.size(2)]).to(mel2.device)

    # Extract style embeddings
    feat2 = torchaudio.compliance.kaldi.fbank(ref_waves_16k,
                                          num_mel_bins=80,
                                          dither=0,
                                          sample_frequency=16000)
    feat2 = feat2 - feat2.mean(dim=0, keepdim=True)
    style2 = campplus_model(feat2.unsqueeze(0))

    # Extract F0 (pitch) information
    F0_ori = f0_fn(ref_waves_16k[0], thred=0.03)
    F0_alt = f0_fn(converted_waves_16k[0], thred=0.03)

    if device.type == "mps":
        F0_ori = torch.from_numpy(F0_ori).float().to(device)[None]
        F0_alt = torch.from_numpy(F0_alt).float().to(device)[None]
    else:
        F0_ori = torch.from_numpy(F0_ori).to(device)[None]
        F0_alt = torch.from_numpy(F0_alt).to(device)[None]

    # Process F0 for pitch adjustment
    voiced_F0_ori = F0_ori[F0_ori > 1]
    voiced_F0_alt = F0_alt[F0_alt > 1]

    log_f0_alt = torch.log(F0_alt + 1e-5)
    voiced_log_f0_ori = torch.log(voiced_F0_ori + 1e-5)
    voiced_log_f0_alt = torch.log(voiced_F0_alt + 1e-5)
    median_log_f0_ori = torch.median(voiced_log_f0_ori)
    median_log_f0_alt = torch.median(voiced_log_f0_alt)

    # Shift alt log f0 level to ori log f0 level
    shifted_log_f0_alt = log_f0_alt.clone()
    if auto_f0_adjust:
        shifted_log_f0_alt[F0_alt > 1] = log_f0_alt[F0_alt > 1] - median_log_f0_alt + median_log_f0_ori
    shifted_f0_alt = torch.exp(shifted_log_f0_alt)
    if pitch_shift != 0:
        shifted_f0_alt[F0_alt > 1] = adjust_f0_semitones(shifted_f0_alt[F0_alt > 1], pitch_shift)

    # Length regulation
    cond, _, codes, commitment_loss, codebook_loss = model.length_regulator(S_alt, ylens=target_lengths, n_quantizers=3, f0=shifted_f0_alt)
    prompt_condition, _, codes, commitment_loss, codebook_loss = model.length_regulator(S_ori, ylens=target2_lengths, n_quantizers=3, f0=F0_ori)
    interpolated_shifted_f0_alt = torch.nn.functional.interpolate(shifted_f0_alt.unsqueeze(1), size=cond.size(1), mode='nearest').squeeze(1)

    # Process in chunks
    max_source_window = max_context_window - mel2.size(2)
    processed_frames = 0
    generated_wave_chunks = []
    progress_bar = st.progress(0)

    # Generate chunk by chunk
    while processed_frames < cond.size(1):
        chunk_cond = cond[:, processed_frames:processed_frames + max_source_window]
        chunk_f0 = interpolated_shifted_f0_alt[:, processed_frames:processed_frames + max_source_window]
        is_last_chunk = processed_frames + max_source_window >= cond.size(1)
        cat_condition = torch.cat([prompt_condition, chunk_cond], dim=1)

        # Update progress
        progress = min(1.0, processed_frames / cond.size(1))
        progress_bar.progress(progress)

        with torch.autocast(device_type=device.type, dtype=torch.float16 if use_fp16 else torch.float32):
            # Voice Conversion
            vc_target = model.cfm.inference(cat_condition,
                                           torch.LongTensor([cat_condition.size(1)]).to(mel2.device),
                                           mel2, style2, None, diffusion_steps,
                                           inference_cfg_rate=inference_cfg_rate)
            vc_target = vc_target[:, :, mel2.size(-1):]

        vc_wave = vocoder_fn(vc_target.float()).squeeze().cpu()
        if vc_wave.ndim == 1:
            vc_wave = vc_wave.unsqueeze(0)

        if processed_frames == 0:
            if is_last_chunk:
                output_wave = vc_wave[0].cpu().numpy()
                generated_wave_chunks.append(output_wave)
                break
            output_wave = vc_wave[0, :-overlap_wave_len].cpu().numpy()
            generated_wave_chunks.append(output_wave)
            previous_chunk = vc_wave[0, -overlap_wave_len:]
            processed_frames += vc_target.size(2) - overlap_frame_len
        elif is_last_chunk:
            output_wave = crossfade(previous_chunk.cpu().numpy(), vc_wave[0].cpu().numpy(), overlap_wave_len)
            generated_wave_chunks.append(output_wave)
            break
        else:
            output_wave = crossfade(previous_chunk.cpu().numpy(), vc_wave[0, :-overlap_wave_len].cpu().numpy(), overlap_wave_len)
            generated_wave_chunks.append(output_wave)
            previous_chunk = vc_wave[0, -overlap_wave_len:]
            processed_frames += vc_target.size(2) - overlap_frame_len

    # Combine all chunks
    progress_bar.progress(1.0)
    final_audio = np.concatenate(generated_wave_chunks)
    return sr, final_audio

def save_audio_to_file(audio_data, sr, format="wav"):
    """Save audio data to a file in the specified format"""
    if format == "wav":
        import scipy.io.wavfile as wavfile
        with BytesIO() as buffer:
            wavfile.write(buffer, sr, (audio_data * 32768.0).astype(np.int16))
            return buffer.getvalue()
    elif format == "mp3":
        with BytesIO() as buffer:
            audio_segment = AudioSegment(
                (audio_data * 32768.0).astype(np.int16).tobytes(),
                frame_rate=sr,
                sample_width=2,
                channels=1
            )
            audio_segment.export(buffer, format="mp3", bitrate="320k")
            return buffer.getvalue()
    else:
        raise ValueError(f"Unsupported format: {format}")

def ensure_wav(file_path):
    if not file_path.lower().endswith('.wav'):
        sound = AudioSegment.from_file(file_path)
        tmp_wav = tempfile.NamedTemporaryFile(delete=False, suffix=".wav")
        sound.export(tmp_wav.name, format="wav")
        return tmp_wav.name
    return file_path

def rename_file_with_uuid(original_path):
    new_filename = f"{uuid.uuid4()}.wav"
    new_path = os.path.join(tempfile.gettempdir(), new_filename)
    shutil.copy(original_path, new_path)
    return new_path

def main():
    # App header
    st.markdown('<div class="stHeader">', unsafe_allow_html=True)
    st.title("🎤 Voice Conversion Studio")
    st.markdown("Transform any voice or singing with zero-shot voice conversion")
    st.markdown('</div>', unsafe_allow_html=True)

    # Sidebar for settings
    with st.sidebar:
        st.markdown('<h2>⚙️ Settings</h2>', unsafe_allow_html=True)

        # Model settings
        st.markdown('#### Model Configuration')
        use_fp16 = st.checkbox("Use FP16 (faster, less precise)", value=True)

        # Conversion settings
        st.markdown('#### Conversion Parameters')
        diffusion_steps = st.slider(
            "Diffusion Steps",
            min_value=1,
            max_value=200,
            value=25,
            help="Higher values = better quality but slower (10 for quick preview, 50-100 for best quality)"
        )

        length_adjust = st.slider(
            "Length Adjustment",
            min_value=0.5,
            max_value=2.0,
            value=1.0,
            step=0.1,
            help="<1.0 speeds up, >1.0 slows down"
        )

        inference_cfg_rate = st.slider(
            "Inference CFG Rate",
            min_value=0.0,
            max_value=1.0,
            value=0.7,
            step=0.1,
            help="Controls influence of the reference on the result"
        )

        auto_f0_adjust = st.checkbox(
            "Auto F0 Adjust",
            value=True,
            help="Automatically adjust pitch to match target voice"
        )

        pitch_shift = st.slider(
            "Pitch Shift (semitones)",
            min_value=-24,
            max_value=24,
            value=0,
            step=1,
            help="Shift pitch up or down by semitones"
        )

        # About section
        st.markdown('<div class="stInfo">', unsafe_allow_html=True)
        st.markdown("#### About")
        st.markdown("""
        This app uses Seed-VC for zero-shot voice conversion.
        Upload any source audio and reference voice to transform.

        For details and updates, check the
        [GitHub repository](https://github.com/Plachta/seed-vc).
        """)
        st.markdown('</div>', unsafe_allow_html=True)

    # Main content area
    col1, col2 = st.columns(2)

    with col1:
        st.markdown("### Source Audio")
        source_file = st.file_uploader("Upload source audio to convert", type=["wav", "mp3", "ogg", "flac"])

        # Examples section for source audio
        st.markdown("#### Or try an example:")
        example_sources = {
            "Male Singing Voice": "examples/source/jay_0.wav",
            "Female Singing Voice": "examples/source/yae_0.wav",
            "English Rap": "examples/source/Wiz Khalifa,Charlie Puth - See You Again [vocals]_[cut_28sec].wav",
            "Techno Vocals": "examples/source/TECHNOPOLIS - 2085 [vocals]_[cut_14sec].wav"
        }

        source_selection = st.selectbox("Select example source", list(example_sources.keys()))
        selected_source = example_sources[source_selection]

        if source_file is not None:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_source:
                tmp_source.write(source_file.getvalue())
                source_path = tmp_source.name
        else:
            # Rename the example file to a temp file with a uuid-based name
            source_path = rename_file_with_uuid(selected_source)

        # Display source audio
        if source_path:
            st.audio(source_path)

    with col2:
        st.markdown("### Reference Voice")
        target_file = st.file_uploader("Upload reference voice", type=["wav", "mp3", "ogg", "flac"])

        # Examples section for reference voice
        st.markdown("#### Or try an example:")
        example_targets = {
            "Male Voice 1": "examples/reference/dingzhen_0.wav",
            "Female Voice 1": "examples/reference/azuma_0.wav",
            "Female Voice 2": "examples/reference/teio_0.wav",
            "Male Voice 2": "examples/reference/trump_0.wav"
        }

        target_selection = st.selectbox("Select example target", list(example_targets.keys()))
        selected_target = example_targets[target_selection]

        if target_file is not None:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_target:
                tmp_target.write(target_file.getvalue())
                target_path = tmp_target.name
        else:
            # Rename the example file to a temp file with a uuid-based name
            target_path = rename_file_with_uuid(selected_target)

        # Display target audio
        if target_path:
            st.audio(target_path)

    # Conversion section
    st.markdown("### 🔄 Voice Conversion")

    # Settings summary
    st.markdown('<div class="stInfo">', unsafe_allow_html=True)
    st.markdown(f"""
    **Current Settings:**
    Diffusion Steps: {diffusion_steps} |
    Length Adjust: {length_adjust} |
    CFG Rate: {inference_cfg_rate} |
    Auto F0: {'✓' if auto_f0_adjust else '✗'} |
    Pitch Shift: {pitch_shift} semitones
    """)
    st.markdown('</div>', unsafe_allow_html=True)

    # Convert button
    if st.button("🎵 Convert Voice", key="convert"):
        if source_path and target_path:
            with st.spinner("Loading models... (first run may take a while)"):
                models = load_models(use_fp16)

            with st.spinner("Converting voice... This may take a few minutes"):
                settings = {
                    "diffusion_steps": diffusion_steps,
                    "length_adjust": length_adjust,
                    "inference_cfg_rate": inference_cfg_rate,
                    "auto_f0_adjust": auto_f0_adjust,
                    "pitch_shift": pitch_shift
                }

                # Perform voice conversion (returns sampling rate and audio data for converted vocals)
                sr, audio_data = convert_voice(models, source_path, target_path, settings)

                # Save the converted vocals as WAV and MP3 (converted vocals only)
                wav_data = save_audio_to_file(audio_data, sr, "wav")
                mp3_data = save_audio_to_file(audio_data, sr, "mp3")

                # ------------------------------
                # DEMUCS SEPARATION & COMBINATION
                # ------------------------------
                # Ensure the source audio is in WAV format for Demucs
                source_wav = ensure_wav(source_path)

                # Run Demucs to separate vocals and no_vocals
                command = f"demucs --two-stems=vocals {source_wav}"
                subprocess.run(command.split(), stdout=subprocess.PIPE)

                # Determine output path from Demucs.
                base = os.path.splitext(os.path.basename(source_wav))[0]
                no_vocals_path = os.path.join("separated", "htdemucs", base, "no_vocals.wav")

                # Load the instrument (no_vocals) track from the separated files
                try:
                    instrument = AudioSegment.from_file(no_vocals_path)
                except Exception as e:
                    st.error(f"Error loading no_vocals file from Demucs: {e}")
                    return

                # Load the converted vocals directly from wav_data (using BytesIO)
                try:
                    vocal = AudioSegment.from_file(BytesIO(wav_data), format="wav")
                except Exception as e:
                    st.error(f"Error loading converted vocals: {e}")
                    return

                # Overlay the converted vocals with the original instruments (no_vocals)
                combined = vocal.overlay(instrument)

                # Export combined audio to a bytes buffer
                combined_out = BytesIO()
                combined.export(combined_out, format='wav')
                combined_bytes = combined_out.getvalue()

                # ------------------------------
                # Display Results
                # ------------------------------
                st.success("✅ Voice conversion complete!")

                st.markdown("### 🔊 Converted Vocals (Voice Conversion Output)")
                st.audio(wav_data, format="audio/wav")

                st.markdown("### 🔊 Final Cover (Combined with Original Instruments)")
                st.audio(combined_bytes, format="audio/wav")

                # Download buttons for converted vocals and final cover
                col1, col2, col3 = st.columns(3)
                with col1:
                    st.download_button(
                        label="⬇️ Download Converted WAV",
                        data=wav_data,
                        file_name="converted_voice.wav",
                        mime="audio/wav"
                    )
                with col2:
                    st.download_button(
                        label="⬇️ Download Converted MP3",
                        data=mp3_data,
                        file_name="converted_voice.mp3",
                        mime="audio/mp3"
                    )
                with col3:
                    st.download_button(
                        label="⬇️ Download Final Cover WAV",
                        data=combined_bytes,
                        file_name="FinalCover.wav",
                        mime="audio/wav"
                    )
        else:
            st.error("Please provide both source and reference audio files")

if __name__ == "__main__":
    main()

Overwriting streamlit_app.py


In [ ]:
%pip install imageio[ffmpeg]
%pip install munch huggingface_hub

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from pyngrok import ngrok

ngrok_key = "2usw6Qj0UF9xT6BpnxOmvvh93dV_3LteB2h5NSRuP8iuygysm"
port = 8501

ngrok.set_auth_token(ngrok_key)
ngrok.connect(port).public_url

In [ ]:
%cd "C:/Users/Welcome/OneDrive/Documents/seed-vc"
!streamlit run streamlit_app.py

C:\Users\Welcome\OneDrive\Documents\seed-vc
